**TDDFT method**
$$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\renewcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

Using the lewenstein integral method one cannot account for the multi-electron contribution when it comes to a molecular system. Hence a quantum description which can include the multielectron dynamics is necessary. One such method is using time dependent density functional theory (TDDFT).

In TDDFT method, using Runge-Gross theorem a one-to-one correspondance between the time dependent potentials and densities is established. Hence the TDKS equation is given by:

$$i\frac{\partial}{\partial~t}\phi_i(r,t) = [-\frac{\nabla^2}{2}+V_{ks}[n](r,t)]\phi_i(r,t)$$

$V_{ks}[n](r,t) = V_{ext}(r,t) + V_H[n](r,t)+ V_{XC}[n](r,t)$: KS potential given by one body external potential, non-interacting Hartree potential and XC potential.

$V_H[n](r,t) = \int dr^\prime \frac{n(r^\prime ,t)}{|r-r^\prime|}$: Hartree potential

$\phi_i(r,t)$: KS orbital

$n(r,t) = \sum_i^{N}|\phi_i(r,t)|^2$: Time dependent densiyt of N-interacting systems

Using the time dependent density, one can calculate the time dependent dipole moment:

$$d(t) = \int dr~r n(r,t) = \sum_j \int dr~r|\phi_j(r,t)|^2$$

Acceleration dipole moment can be calculated using the above time dependent dipole moment.

$$a(t) = \frac{d^2}{dt^2}d(t) = -\int dr~n(r,t)\nabla~V_{ks}(r,t)$$

Taking the fourier transform of the induced dipole moment modulus squared we can calculate the harmonic generation dipole power spectrum.

$$P(\omega) = \frac{1}{t_f^2}\begin{vmatrix}\int_0^{t_f}dt~e^{-i\omega~t}d(t)\end{vmatrix}^2$$

Similarly, taking the fourier transform of the acceleration dipole moment, one get the harmonic spectrum 
in the direction of polarization
$$H(\omega) = \frac{1}{t_f^2}\begin{vmatrix}\int_0^{t_f}dt~e^{-i\omega~t}a(t)\end{vmatrix}^2$$

In this tutorial we will first calculate the time dependent dipole moment using TDDFT method implemented in [Octopus code](https://octopus-code.org/wiki/Main_Page)

First we begin with calculation of the ground state properties for $N_2$ molecule using LDA method. 


Let's start with some technical code.

## Technical prelude

In [2]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from scipy.fftpack import fft, fftfreq
from scipy import constants as const
from ase import io, Atom, Atoms

In [3]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120   # make figures larger

In [4]:
notebook_path = os.getcwd()
print(notebook_path)

/home/nagyg/Projects/ElissNotebook/Codes/ASE_Octopus_Demo


If your Octopus binaries are in a local folder, add it to `$PATH` here. If they are already there, you can skip the cell below. This works for Linux; on Windows, you have to make sure that Octopus is visible for the system yourself.

In [ ]:
Octopus_bin_path = os.environ["HOME"]+"/.local/Octopus/bin"
os.environ['PATH']= ":".join([Octopus_bin_path, os.environ['PATH']])

This folder will be used for the files created while running the simulations.

In [ ]:
tmpdir=os.path.abspath(os.path.join(notebook_path, 'calc_atomic_Xe'))
print('Created calculation directory: '+ tmpdir)
os.makedirs(tmpdir,exist_ok=True)
os.chdir(tmpdir)

Created calculation directory: /home/nagyg/Projects/ElissNotebook/Codes/ASE_Octopus_Demo/calc_atomic_Xe


## Ground state calculation on N2

The input file for the ground state calculation is given in directory under the name test_octopus. 

OR

You can create this directory here:

In [5]:
# Make new directory
Oct_dir = os.path.join(notebook_path, 'test_octopus')
if not os.path.isdir(Oct_dir):
    Oct_dir = os.makedirs(Oct_dir)
else:
    print('Using existing directory ' + Oct_dir)

Using existing directory /home/nagyg/Projects/ElissNotebook/Codes/ASE_Octopus_Demo/test_octopus


We write the geometrical coordinates of the molecule in the file: N2.xyz
```
2

N      -0.66044100      -0.00736549       0.00000000
N       0.29863339       0.51058848       0.00000000
```

In [4]:
with open(Oct_dir+'\\N2.xyz','w') as N2_geo:
    N2_geo.write('2 \n \n N -0.66044100 -0.00736549 0.00000000 \n N 0.29863339 0.51058848 0.00000000')

Octopus code executes the command with input file name as 'inp'. Hence, for ground state calculations we copy the file 'N2.gs.inp' to 'inp'.

The input file:

**#This test calculates a three-dimensional model for nitrogen molecule, considering**

CalculationMode = gs

Dimensions = 3

fromScratch = yes

**#Shape of the simulation box**

BoxShape = sphere

**#In the original reference, the spacing is 0.25 a.u.**

Spacing = 0.35

**#In the original reference, the box is 300 a.u. long. p**

Radius = 200.0

**#We use Kohn Sham method**

TheoryLevel = kohn_sham

**#Coordinates for N2 molecule**

XYZCoordinates = "N2.xyz"

**#Maximum number of iterations for SCF calculations**

MaximumIter = 300

**#Boundaries of the simulation box absorbing defined with mask function.**

AbsorbingBoundaries = mask

**#Specifies the boundary width**

AbWidth = 50




In [5]:
import shutil


Lines = ['# This test calculates a one-dimensional model for Hydrogen, considering \n',
         'CalculationMode = gs \n',
         'Dimensions = 3 \n',
         'fromScratch = yes \n',
         'BoxShape = sphere \n',
         '# In the original reference, the spacing is 0.25 a.u. \n',
         'Spacing = 0.35 \n',
         '# In the original reference, the box is 300 a.u. long. p \n',
         'Radius = 200.0 \n',
         '#We use Kohn Sham method \n',
         'TheoryLevel = kohn_sham \n',
         '#Coordinates for N2 molecule \n',
         'XYZCoordinates = "N2.xyz" \n',
         '#Maximum number of iterations for SCF calculations \n',
         'MaximumIter = 300 \n',
         '#Boundaries of the simulation box absorbing defined with mask function. \n',
         'AbsorbingBoundaries = mask \n',
         '#Specifies the boundary width \n',
         'AbWidth = 50 \n']
try:
    shutil.copyfile(Oct_dir+'\\N2.gs.inp', Oct_dir+'\\inp')
except:
    with open(Oct_dir+'\\N2.gs.inp','w') as gs_inp:
        gs_inp.writelines(Lines)
    shutil.copyfile(Oct_dir+'\\N2.gs.inp', Oct_dir+'\\inp')

To get more info about the input parameters, you can use the `oct-help` command-line utility:

In [1]:
parameter_to_lookup = "Spacing"
os.system(f"oct-help -p " + parameter_to_lookup)

Variable: Spacing
Type:     float
Section:  Mesh


0

Description:
    The  spacing  between  the points in the mesh. This controls the quality of
    the  discretization:  smaller  spacing  gives  more  precise  results  but
    increased computational cost.
    
    When  using  curvilinear coordinates, this is a canonical spacing that will
    be  changed  locally  by  the  transformation. In periodic directions, your
    spacing may be slightly different than what you request here, since the box
    size must be an integer multiple of the spacing.
    
    The  default  value  is  defined  by  the  species  if  only  default
    pseudopotentials  are  used  or  by  the  image  resolution  if  BoxShape =
    box_image. Otherwise, there is no default.
    
    It  is  possible  to  have a different spacing in each one of the Cartesian
    directions if we define Spacing as block of the form
    
    %Spacing
      spacing_x | spacing_y | spacing_z
    %


To execute octopus calculations you can load the octopus modeule or call 
octopus from the directory where octopus is installed and execute in terminal:

> octopus | tee gs.log 

(the `tee` command is just a technical help to print the output both to a file and to STDOUT.)

In [ ]:
os.system("octopus | tee gs.log")

The output file for the ground state calculations is given in test_octopus directory as gs.log

This calculation takes about 32 mins when run in serial.

The ouptup of the calculation will give several SCF iterations and end with:

#State  Eigenvalue (H) Occupation    Error

      1       -1.045563    2.000000   (2.6E-05)
      
      2       -0.505260    2.000000   (2.0E-05)
      
      3       -0.470952    2.000000   (3.8E-05)
      
      4       -0.406124    2.000000   (2.7E-06)
      
      5       -0.359920    2.000000   (3.1E-05)


The ionization potential of 0.36 Hartree (9.80eV) is a poor match for experimental value of [15.6eV](https://webbook.nist.gov/cgi/cbook.cgi?ID=C7727379&Mask=20). However, quantitative results is beyond the scope of this tutorial. The ionization potential value can be improved by using different method for calculation (see [M.R. Mack et al. 2013](https://doi.org/10.1016/j.cplett.2012.11.045)).

Now we run the time dependent (td) calculations:

The input file of the td calculations is given as 'N2.td.inp' which is present in the test directory. If not one can create using:
```
#This test calculates a three-dimensional model for nitrogen molecule, considering
CalculationMode = td
Debug = trace
Dimensions = 3
fromScratch = yes
PseudopotentialSet = pseudodojo_pbe
FilterPotentials = filter_none
XYZCoordinates = "N2.xyz"
BoxShape = sphere
Radius = 50*angstrom
Spacing = 0.3*angstrom

TheoryLevel = kohn_sham
#Frequency corresponding to 800nm.
omega = 0.05655
period = 0.6*femtosecond #*pi/omega

#In the original reference, there are 96 cycles instead of 8.
stime = 30*period

#The time-step is shorter in the original reference. But the propagation
#algorithm is also probably different.
dt = period/100
TDPropagationTime = stime
TDPropagator = exp_mid
TDExponentialMethod = lanczos
TDExpOrder = 20
TDTimeStep = dt
%TDExternalFields
electric_field   | 1 | 0 | 0 | omega | "envelope_function"
%
electric_amplitude = (sqrt(4*10^14)/sqrt(3.509470*10^16))
%TDFunctions
"envelope_function" | tdf_from_expr | "electric_amplitude*(sin(pi/stime*t))^2"
%
#AbsorbingBoundaries = cap
AbsorbingBoundaries = mask
AbWidth = 50
%TDOutput 
 laser
 multipoles 
 dipole_acceleration
%
```

In [6]:
Lines_td = ['#This test calculates a three-dimensional model for nitrogen molecule, considering \n',
            'CalculationMode = td \n',
            'Debug = trace \n',
            'Dimensions = 3 \n',
            'fromScratch = yes \n',
            'PseudopotentialSet = pseudodojo_pbe \n',
            'FilterPotentials = filter_none \n',
            'XYZCoordinates = "N2.xyz" \n',
            'BoxShape = sphere \n',
            'Radius = 50*angstrom \n',
            'Spacing = 0.3*angstrom \n',
            'TheoryLevel = kohn_sham \n',
            '#Frequency corresponding to 800nm. \n',
            'omega = 0.05655 \n',
            'period = 0.6*femtosecond \n',
            '#In the original reference, there are 96 cycles instead of 8. \n',
            'stime = 30*period \n',
            '#The time-step is shorter in the original reference. But the propagation \n',
            '#algorithm is also probably different. \n',
            'dt = period/100 \n',
            'TDPropagationTime = stime \n',
            'TDPropagator = exp_mid \n',
            'TDExponentialMethod = lanczos \n',
            'TDExpOrder = 20 \n',
            'TDTimeStep = dt \n',
            '%TDExternalFields \n',
            'electric_field   | 1 | 0 | 0 | omega | "envelope_function" \n',
            '% \n',
            'electric_amplitude = (sqrt(4*10^14)/sqrt(3.509470*10^16)) \n',
            '%TDFunctions \n',
            '"envelope_function" | tdf_from_expr | "electric_amplitude*(sin(pi/stime*t))^2" \n',
            '% \n',
            'AbsorbingBoundaries = mask \n',
            'AbWidth = 50 \n',
            '%TDOutput \n',
            ' laser \n',
            ' multipoles \n',
            ' dipole_acceleration \n',
            '%']

In [7]:
try:
    shutil.copyfile(Oct_dir+'\\N2.td.inp', Oct_dir+'\\inp')
except:
    with open(Oct_dir+'\\N2.td.inp','w') as td_inp:
        td_inp.writelines(Lines_td)
    shutil.copyfile(Oct_dir+'\\N2.td.inp', Oct_dir+'\\inp')

To execute octopus td calculations you can execute in terminal:

> octopus | tee td.log 

This takes a long time though, so during the lesson, we will use the result of an already executed calculation. Feel free to run this calculation outside the lesson.

In [ ]:
RUN_TD = False
if RUN_TD:
    os.system('octopus | tee td.log')

To generate the files that can calculate the fourier transform of the time dependent dipole one can execute in terminal:
> oct-harmonic-spectrum -m 1


In [6]:
os.system('oct-harmonic-spectrum -m 1')

*** Fatal Error (description follows)
Error initializing parser
Cannot open input file!
Please provide an input file with name inp in the current workdir


0

This command will generate hs-mult.x file. 

The hs-mult.x file contains:
#w                  H(w) 

#H             (b/hbar/H^2 

   0.000000E+00   0.335852E-03
   
   0.514091E-02   0.402734E-03
   
   0.102818E-01   0.863577E-03
   
   0.154227E-01   0.328424E-04
   
$\dots \dots$ 

On plotting the hs-mult.x file we get the harmonic generation dipole power spectrum.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

hs_file = Oct_dir+'\\hs-mult.x'
df = pd.read_csv(hs_file,header=None,skiprows=2, delim_whitespace=True)
plt.semilogy(df[0]/0.056,df[1],label='N2')
plt.xlabel('Harmonic order')
plt.ylabel('Harmonic intensity (arb. units)')

ModuleNotFoundError: No module named 'pandas'

# TDDFT on a single atom, and comparison with the Lewenstein model

In the previous section, we saw how to run an Octopus calculation on an $N_2$ molecule. In a previous notebook we also encountered the Lewenstein model, which only works forr single atoms. To compare, we now run the previous workflow on an isolated Xenon atom.

We also (briefly) introduce a powerful Python tool, ASE, that lets us load or define molecules and periodic structures. We only use it to define the atomic coordinates, but it can be used for much more - in fact, it could also interface to the Octopus code and manage the calculation itself.

In this case, we also show how to obtain the harmonic spectrum from the dipole moment itself, using our own postprocessing implementation instead of `oct-harmonic-spectrum`.